# Proyecto investigación operativa

Tenemo un estadio de fútbol y queremos ubicar los asientos conforme obtengamos la mayor cantidad de beneficio posible. Los precios de las entradas van acorde a como de bien se ve desde cada asiento. Por tanto, las de tribuna y preferencia son las más caras, valiendo tribuna 20 euros, y preferencia 15. Mientras que las de pabellón y y maratón son de 10 y 5 eureos respectivamente.

El número de entradas de las que disponemos son 100 para todo el estadio. Pero hay que tener en cuenta que las de maratón son 2 veces más vendidas que las de tribuna, salvo por como máximo 10 entradas.

#### Variables de Decisión
$x_1$: Número de entradas vendidas de Tribuna.
$x_2$: Número de entradas vendidas de Preferencia.
$x_3$: Número de entradas vendidas de Pabellón.
$x_4$: Número de entradas vendidas de Maratón.

#### Función Objetivo
Maximizar el beneficio total:
$$
Z = 20x_1 + 15x_2 + 10x_3 + 5x_4
$$

#### Restricciones
1. Restricción del número total de entradas:
$$
x_1 + x_2 + x_3 + x_4 \leq 100
$$

2. Restricción de las entradas de Maratón:
$$
x_1 - 2x_4\leq  10
$$

3. Restricción de no negatividad:
$$
x_1 , x_2 , x_3 , x_4 \geq 0
$$


In [ ]:
# Importar Pyomo
from pyomo.environ import *

# Crear el modelo
model = ConcreteModel()

# Definir las variables
model.x1 = Var(within=NonNegativeReals)
model.x2 = Var(within=NonNegativeReals)

# Definir la función objetivo
model.obj = Objective(expr=40*model.x1 + 30*model.x2, sense=maximize)

# Definir las restricciones
model.constraint1 = Constraint(expr=2*model.x1 + 3*model.x2 <= 60)
model.constraint2 = Constraint(expr=model.x1 + 2*model.x2 <= 40)

# Resolver el problema
solver = SolverFactory('glpk')
solver.solve(model)

# Mostrar los resultados
model.x1.display()
model.x2.display()
model.obj.display()